#### Titanic adventure

In [26]:
# get data

import pandas as pd
import numpy as np

# visualizations
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
plt.rcParams['figure.figsize'] = 12,8

In [27]:
train = pd.read_csv('titanic data/train.csv')
test = pd.read_csv('titanic data/test.csv')

train = train.drop(['PassengerId', 'Ticket', 'Name'], axis=1)
test = test.drop(['PassengerId', 'Ticket', 'Name'], axis=1)

In [23]:
train.describe()
print(train.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


#### checking NA, over 70% in train for cabin is NA - remove

In [31]:
train.isna().sum()
print(train.shape)

(891, 9)


In [34]:
if 'Cabin' in train.columns:
    train = train.drop(['Cabin'], axis=1)

if 'Cabin' in test.columns:
    test = test.drop(['Cabin'], axis=1)

#### before imputer: label encoding

###### impute data for all missing column
* instead of doing this separately for each columns - single.impute

label encoder (ordinar?) vs onehotencoder
* first keeps one column, just 1/2/3/4/5..
* second creates n dummy columns

depending on situation dummy might be better.
Example:
* when we have bad/medium/string/superstrong etc - it most case there is no differecnce betwwen best and worse vs medium and worse - just different labels. But for ordinal encoder program might think that it is better to use 3/4/5 ... - > higher grades

---
so in some cases/ most? one hot encoder -> full encoder

#### train['Sex'] != train[['Sex']] !!


In [48]:
from sklearn.preprocessing import OneHotEncoder
# for it ofc we need categorical staff
z = OneHotEncoder()
encoder_result = z.fit_transform(train[['Sex']])
encoder_result.toarray()
print(z.categories_)

[array(['female', 'male'], dtype=object)]


In [36]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
imputer.fit(train)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'male'

In [52]:
## numerical pipeline - imputer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

num_pipeline.fit_transform(train[['Age']])[0] 

array([22.])

In [55]:
# categorical encoder - onehotencoding

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

cat_pipeline.fit_transform(train[['Sex']]).toarray()[0]

array([0., 1.])

In [69]:
## combine:

from sklearn.compose import ColumnTransformer

numerical_columns = train.select_dtypes(include='number').columns
categorical_columns = list(set(train.columns) - set(numerical_columns))
print(numerical_columns)
print(categorical_columns)

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, numerical_columns),
    ("cat", cat_pipeline, categorical_columns)
])

train_cleaned = full_pipeline.fit_transform(train)

Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
['Embarked', 'Sex']


## TASK:
GET IN BACK COLUMN NAMES

In [71]:
pd.DataFrame(train_cleaned)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0,0.0,1.0,0.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,2.0,27.0,0.0,0.0,13.0000,0.0,0.0,1.0,0.0,0.0,1.0
887,1.0,1.0,19.0,0.0,0.0,30.0000,0.0,0.0,1.0,0.0,1.0,0.0
888,0.0,3.0,28.0,1.0,2.0,23.4500,0.0,0.0,1.0,0.0,1.0,0.0
889,1.0,1.0,26.0,0.0,0.0,30.0000,1.0,0.0,0.0,0.0,0.0,1.0
